In [30]:
from poloniex import Poloniex

polo = Poloniex()

In [31]:
markets = polo.returnTicker()
pairs = []
for name, thinks in markets.items():
    pairs.append(name)

In [32]:
# funciones necesarias para el manejo del tiempo
from datetime import datetime, timedelta
from time import sleep
from tusp import string2ts
from bot import prepareData
import numpy as np
import pylab as pl
%matplotlib inline

class PairData:
    
    def __init__(self, pair, period, weeks = 4.3*3, backto = None):
            
        self.pair = pair
        self.period = period
        delta = timedelta(weeks = weeks)
        self.delta = delta
        start = ""
        end = ""

        # Sí el tiempo final es diferente al actual...
        if backto != None:
            to = datetime.strptime(backto, "%Y-%m-%d %H:%M:%S")
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
            
            tf = to + delta
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
        else:
            tf = datetime.now()
            self.tf = tf
            end = string2ts(tf.strftime('%Y-%m-%d %H:%M:%S'))
            
            to = tf-delta
            self.to = to
            start = string2ts(to.strftime('%Y-%m-%d %H:%M:%S'))
                              

        self.start = start
        self.end = end
        df = prepareData(pair=pair, start=start, end=end, period=int(period))
        
        self.df = df
        self.market_return()
        self.cum_volume()
    
    def market_return(self):
        """Calcula el retorno del mercado vía log_retornos
        """
        
        serie = self.df["close"]

        # calculando los log retornos
        log_return = np.log(serie).diff().fillna(0)

        # calculando log retorno acumulado
        cum_logr = log_return.cumsum()
        cum_r = np.exp(cum_logr) - 1
        
        self.l_return = log_return
        self.m_return = cum_r
        
    def cum_volume(self):
        """Calcula el volumen acumulado del mercado
        """
        serie = self.df["volume"]
        self.c_volume = serie.cumsum()[-1]
        
    def sortino(self, log = False):
        """ Calcula en ratio de Sortino
        
        """
        
        if log:
            rets = self.l_return
         
        else:
            rets = self.m_return

        semi_var = rets[rets < 0] ** 2
        semi_var = semi_var.sum()/len(rets)
        sortino = np.sqrt(semi_var)

        return rets.mean()/sortino

    def sharpe(self, period_ret = 0.2):
        """
        Calculate the annualised Sharpe ratio of a returns stream 
        based on a number of trading periods, N. N defaults to 252,
        which then assumes a stream of daily returns.

        The function assumes that the returns are the excess of 
        those compared to a benchmark.
        """
        N = self.delta.days
        returns = self.l_return
        excess_ret = returns - period_ret/N
        return np.sqrt(N) * excess_ret.mean() / excess_ret.std()
    
dgb = PairData("BTC_DGB", 3600*4)

print dgb.c_volume

69405.241737


In [33]:
# cálculo del ratio de Sortino del último mes
volumes = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2)
    volume = asset.c_volume
    volumes.append([pair, volume])
    #print sortino

volumes.sort(key = lambda l:l[1], reverse=True)

In [34]:
print volumes[:6]

[[u'BTC_XRP', 645479.59457882994], [u'BTC_ETH', 517940.85817626998], [u'BTC_BCH', 259859.07800493002], [u'BTC_STR', 255435.18804196999], [u'BTC_LTC', 191898.49865677007], [u'BTC_XMR', 103318.25899023995]]


In [35]:
# cálculo del ratio de Sortino del último mes
sortinos = []
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]
for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2)
    sortino = asset.sortino(log = True)
    sortinos.append([pair, sortino])
    #print sortino

sortinos.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos2 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2, "2017-11-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos2.append([pair, sortino])
    #print sortino

sortinos2.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sortino del penúltimo mes
sortinos3 = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*4, 4.28*2, "2017-10-14 00:00:00")
    sortino = asset.sortino(log = True)
    sortinos3.append([pair, sortino])
    #print sortino

sortinos3.sort(key = lambda l:l[1], reverse=True)

In [43]:
print "Octubre a Noviembre"
print sortinos3[:3]
print "Noviembre a Diciembre"
print sortinos2[:3]
print "Diciembre a Enero"
print sortinos[:3]

Octubre a Noviembre
[[u'BTC_STR', 0.18996736555236127], [u'BTC_SC', 0.16927471652832185], [u'BTC_BURST', 0.15335356963507588]]
Noviembre a Diciembre
[[u'BTC_STR', 0.1960905926315005], [u'BTC_SC', 0.16516678529416567], [u'BTC_BURST', 0.15996807743790623]]
Diciembre a Enero
[[u'BTC_STR', 0.18996736555236127], [u'BTC_SC', 0.16927471652832185], [u'BTC_BURST', 0.15335356963507588]]


In [37]:
# cálculo del ratio de Sharpe del último mes
sharpes = []
for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28*2, "2017-11-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28*2)
    sharpe = asset.sharpe(ret)
    sharpes.append([pair, sharpe])

sharpes.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes2 = []

for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28*2, "2017-10-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28*2, "2017-11-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes2.append([pair, sharpe])

sharpes2.sort(key = lambda l:l[1], reverse=True)

# Cálculo del ratio de Sharpe del penúltimo mes
sharpes3 = []

for pair in pairs:
    asset_past = PairData(pair, 3600*4, 4.28*2, "2017-09-14 00:00:00")
    ret = asset_past.m_return[-1]
    asset = PairData(pair, 3600*4, 4.28*2, "2017-10-14 00:00:00")
    sharpe = asset.sharpe()
    sharpes3.append([pair, sharpe])

sharpes3.sort(key = lambda l:l[1], reverse=True)

In [42]:
print "Octubre y Noviembre"
print sharpes3[:3]
print "Noviembre y Diciembre"
print sharpes2[:3]
print "Diciembre y Enero"
print sharpes[:3]

Octubre y Noviembre
[[u'BTC_EMC2', 0.15729527376916327], [u'BTC_NXT', 0.023479674988932761], [u'BTC_STR', -0.075880999224661863]]
Noviembre y Diciembre
[[u'BTC_STR', 0.45279572024813147], [u'BTC_SC', 0.32720856845234464], [u'BTC_BURST', 0.25713595584531107]]
Diciembre y Enero
[[u'BTC_VTC', 0.76318594268145801], [u'BTC_XBC', 0.41926137632494292], [u'BTC_BCH', 0.41597933973769302]]


In [39]:
sortinos3 = [x for x in sortinos if x[0].split("_")[0] == "BTC"]
print sortinos3[0:7]

[[u'BTC_STR', 0.18996736555236127], [u'BTC_SC', 0.16927471652832185], [u'BTC_BURST', 0.15335356963507588], [u'BTC_XRP', 0.14252262782046513], [u'BTC_ZRX', 0.1368647096163062], [u'BTC_DOGE', 0.13672545844392933], [u'BTC_DGB', 0.13214376559913726]]


In [40]:
pairs = [x for x in pairs if x.split("_")[0] == "BTC"]

In [41]:
from indicators import rsi
rsi_list = []

for pair in pairs:
    #print "\n\t",pair
    asset = PairData(pair, 3600*24, weeks = 4.3*6)
    RSI1, RSI2 = rsi(asset.df["close"])
    rsi_list.append([pair, RSI1[-1]])
    #print sortino

rsi_list.sort(key = lambda l:l[1], reverse=False)

print rsi_list[0:6]

[[u'BTC_NXT', 40.532260289689546], [u'BTC_BTS', 41.738454386648755], [u'BTC_VTC', 41.875755753143629], [u'BTC_SBD', 42.148668170862223], [u'BTC_BTCD', 43.244597916653454], [u'BTC_EMC2', 44.613689029544055]]
